# NLP: Sentiment Analysis
Choosing and training a model to perform sentiment analysis on catalan text.

In [3]:
import pandas as pd
import numpy as np

from datasets import Dataset, concatenate_datasets

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

/opt/homebrew/anaconda3/envs/andorra-sentiment-analyst/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
import torch

## The data

Check out `data-preprocessing.ipynb` for details about how this initial dataset was chosen, and how the subset dataset was labeled.

In [5]:
comments = pd.read_csv("comments.csv")

In [6]:
labeled = pd.read_csv("manual-labeling/comments_subset_labeled.tsv", sep="\t")

### Defined the labeled dataset

In [6]:
comments_labeled = comments.join(labeled.set_index('comment_id'), on='comment_id', rsuffix='labeled')

In [7]:
comments_labeled = comments_labeled[~comments_labeled['label'].isnull()]

In [8]:
labels = {'positive':       0,
          'neutral':        1,
          'negative':       2,
          'very-negative':  3}

In [9]:
comments_labeled['labels'] = comments_labeled['label'].apply(lambda text_label: labels[text_label])

In [10]:
comments_labeled = comments_labeled[['input_text', 'labels']]

In [11]:
comments_labeled.head()

,input_text,labels
5,[ARTICLE TITLE] Pas endavant del conveni del C...,2
9,[ARTICLE TITLE] Espot confia en el suport d’Es...,3
15,[ARTICLE TITLE] Més de 174.000 euros per a un ...,3
22,[ARTICLE TITLE] Més de 174.000 euros per a un ...,2
25,[ARTICLE TITLE] Més de 174.000 euros per a un ...,2


In [12]:
comments_labeled['labels'].value_counts()

labels
2    114
1     59
3     53
0     24
Name: count, dtype: int64

In [13]:
num_custom_labels = 4

### Define the unlabeled dataset

In [14]:
comments['labels'] = np.nan

In [15]:
len(comments)

2413

In [16]:
comments_unlabeled = comments[~comments['comment_id'].isin(labeled['comment_id'])][['input_text']]

In [17]:
len(comments_unlabeled)

2163

In [18]:
comments_unlabeled.head()

,input_text
0,[ARTICLE TITLE] SDP advoca per un espai transv...
1,[ARTICLE TITLE] SDP advoca per un espai transv...
2,[ARTICLE TITLE] Acord en cercar “consens” i “c...
3,[ARTICLE TITLE] Acord en cercar “consens” i “c...
4,[ARTICLE TITLE] Acord en cercar “consens” i “c...


## Prepare the dataset for the model

### Splitting into training and testing

In [19]:
train, val = train_test_split(
    comments_labeled,
    test_size=0.2,
    random_state=42,
    stratify=comments_labeled['labels']  # Ensures proportional class distribution
)

In [20]:
print(f"Length of training dataset: {len(train)}\nLength of validation dataset: {len(val)}")
print(f"Total length of dataset: {len(train) + len(val)}")

Length of training dataset: 200
Length of validation dataset: 50
Total length of dataset: 250


## Fine-tuning

We'll get the Bert model and Tokenizer from the transformers package.

In [21]:
mbert = 'bert-base-multilingual-cased'
tokenizer = BertTokenizer.from_pretrained(mbert)
model = BertForSequenceClassification.from_pretrained(mbert, num_labels=num_custom_labels)

/opt/homebrew/anaconda3/envs/andorra-sentiment-analyst/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### Defining the datasets in the correct HuggingFace format

In [22]:
train_dataset = Dataset.from_pandas(train)
val_dataset = Dataset.from_pandas(val)

### Tokenize the text

In [23]:
def tokenize(record):
    return tokenizer(record['input_text'], padding='max_length', truncation=True, max_length=128)

In [24]:
train_dataset

Dataset({
    features: ['input_text', 'labels', '__index_level_0__'],
    num_rows: 200
})

In [25]:
train_dataset = train_dataset.map(tokenize, batched=True)
val_dataset = val_dataset.map(tokenize, batched=True)

Map: 100%|██████████| 50/50 [00:00<00:00, 3581.94 examples/s]


### Rename the columns

### Set format for PyTorch

In [26]:
train_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'labels'])
val_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'labels'])

In [27]:
train_dataset

Dataset({
    features: ['input_text', 'labels', '__index_level_0__', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 200
})

### Define training arguments

In [28]:
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=15,
    learning_rate=3e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    warmup_steps=50,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    save_total_limit=2,  # Limit the total amount of checkpoints
    seed=42
)

/opt/homebrew/anaconda3/envs/andorra-sentiment-analyst/lib/python3.12/site-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


### Define metrics

In [29]:
def compute_metrics(p):
    preds = np.argmax(p.predictions, axis=1)
    return {"accuracy": accuracy_score(p.label_ids, preds)}

### Initialize the trainer

In [30]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)

### Training the model !

In [31]:
trainer.train()

  5%|▌         | 10/195 [00:08<02:29,  1.24it/s]

{'loss': 1.3925, 'grad_norm': 4.255208969116211, 'learning_rate': 6e-06, 'epoch': 0.77}


                                                
  7%|▋         | 13/195 [00:11<02:15,  1.34it/s]

{'eval_loss': 1.325324296951294, 'eval_accuracy': 0.46, 'eval_runtime': 0.6519, 'eval_samples_per_second': 76.701, 'eval_steps_per_second': 1.534, 'epoch': 1.0}


 10%|█         | 20/195 [00:19<02:37,  1.11it/s]

...

                                                 
100%|██████████| 195/195 [03:15<00:00,  1.36it/s]

{'eval_loss': 1.6514149904251099, 'eval_accuracy': 0.54, 'eval_runtime': 0.6584, 'eval_samples_per_second': 75.939, 'eval_steps_per_second': 1.519, 'epoch': 15.0}


100%|██████████| 195/195 [03:18<00:00,  1.02s/it]

{'train_runtime': 198.4189, 'train_samples_per_second': 15.12, 'train_steps_per_second': 0.983, 'train_loss': 0.6899081162917309, 'epoch': 15.0}


TrainOutput(global_step=195, training_loss=0.6899081162917309, metrics={'train_runtime': 198.4189, 'train_samples_per_second': 15.12, 'train_steps_per_second': 0.983, 'total_flos': 197336835072000.0, 'train_loss': 0.6899081162917309, 'epoch': 15.0})

### Evalutaing the model

In [32]:
results = trainer.evaluate(eval_dataset=val_dataset)
print("Evaluation Results:", results)

100%|██████████| 1/1 [00:00<00:00, 439.38it/s]

Evaluation Results: {'eval_loss': 1.6218115091323853, 'eval_accuracy': 0.56, 'eval_runtime': 0.6848, 'eval_samples_per_second': 73.012, 'eval_steps_per_second': 1.46, 'epoch': 15.0}


### Save the trained model

In [33]:
# trainer.save_model('./fine-tuned-mbert-sentiment')

# Pseudo-labeling on Unlabeled Data

In [34]:
len(comments_unlabeled)

2163

### Define the Dataset

In [35]:
unlabeled_dataset = Dataset.from_pandas(comments_unlabeled)

### Tokenize

In [36]:
unlabeled_dataset = unlabeled_dataset.map(tokenize, batched=True)

Map: 100%|██████████| 2163/2163 [00:00<00:00, 4114.04 examples/s]


### Define torch format

In [37]:
unlabeled_dataset.set_format('torch', columns=['input_ids', 'attention_mask'])

### Predict the labels for the unlabeled data

In [38]:
predictions = trainer.predict(unlabeled_dataset)
probs = torch.nn.functional.softmax(torch.tensor(predictions.predictions), dim=1)
max_probs, pseudo_labels = torch.max(probs, dim=1)

100%|██████████| 34/34 [00:28<00:00,  1.18it/s]


In [39]:
probs = max_probs.numpy()
pseudo_labels = pseudo_labels.numpy()

### Define confidence threshold

In [40]:
confidence_threshold = 0.9
confident_indices = np.where(probs > confidence_threshold)[0]

In [41]:
print(f"Confident pseudo-labeled samples: {len(confident_indices)}")

Confident pseudo-labeled samples: 847


### Select confident pseudo-labeled data

In [42]:
pseudo_labeled_dataset = unlabeled_dataset.select(confident_indices)
pseudo_labeled_dataset = pseudo_labeled_dataset.add_column('labels', pseudo_labels[confident_indices])

Flattening the indices: 100%|██████████| 847/847 [00:00<00:00, 26281.50 examples/s]


# Combining labeled and pseudo-labeled dataset

In [43]:
expanded_train_dataset = concatenate_datasets([train_dataset, pseudo_labeled_dataset])

In [44]:
print(f"Expanded training samples: {len(expanded_train_dataset)}")

Expanded training samples: 1047


# Fine tune on expanded dataset

In [45]:
trainer_expanded = Trainer(
    model=model,
    args=training_args,
    train_dataset=expanded_train_dataset,
    eval_dataset=val_dataset,  # Use the same validation set
    compute_metrics=compute_metrics,
)

In [46]:
trainer_expanded.train()

  1%|          | 10/990 [00:08<13:56,  1.17it/s]

{'loss': 0.1669, 'grad_norm': 3.2703919410705566, 'learning_rate': 6e-06, 'epoch': 0.15}


  2%|▏         | 20/990 [00:17<13:36,  1.19it/s]

{'loss': 0.1301, 'grad_norm': 1.9483102560043335, 'learning_rate': 1.2e-05, 'epoch': 0.3}


  3%|▎         | 30/990 [00:25<14:21,  1.11it/s]

...


100%|██████████| 990/990 [18:52<00:00,  1.01s/it]

{'eval_loss': 4.027500629425049, 'eval_accuracy': 0.54, 'eval_runtime': 0.6889, 'eval_samples_per_second': 72.579, 'eval_steps_per_second': 1.452, 'epoch': 15.0}


100%|██████████| 990/990 [18:55<00:00,  1.15s/it]

{'train_runtime': 1135.4296, 'train_samples_per_second': 13.832, 'train_steps_per_second': 0.872, 'train_loss': 0.06344544435851276, 'epoch': 15.0}


TrainOutput(global_step=990, training_loss=0.06344544435851276, metrics={'train_runtime': 1135.4296, 'train_samples_per_second': 13.832, 'train_steps_per_second': 0.872, 'total_flos': 1033058331601920.0, 'train_loss': 0.06344544435851276, 'epoch': 15.0})

In [47]:
results = trainer.evaluate(eval_dataset=val_dataset)
print("Evaluation Results:", results)

100%|██████████| 1/1 [00:00<00:00, 530.99it/s]

Evaluation Results: {'eval_loss': 3.2418322563171387, 'eval_accuracy': 0.56, 'eval_runtime': 0.7452, 'eval_samples_per_second': 67.098, 'eval_steps_per_second': 1.342, 'epoch': 15.0}


In [50]:
# trainer_expanded.save_model('./fine-tuned-mbert-sentiment-expanded')

In [1]:
model.save_pretrained("../model")
tokenizer.save_pretrained("../model")

NameError: name 'model' is not defined